In [1]:
!pip install google-api-python-client pandas
from googleapiclient.discovery import build
import pandas as pd
import os

In [2]:

# Replace 'YOUR_API_KEY' with your actual YouTube Data API v3 key
API_KEY = 'yourAPI'
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

In [5]:
def youtube_search(query, max_results=100000):
    all_videos = []
    next_page_token = None
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=API_KEY)

    while len(all_videos) < max_results:
        # Call the search.list method to retrieve results matching the specified query term, sorted by date.
        search_response = youtube.search().list(
            q=query,
            part='id,snippet',
            maxResults=50,  # API's max limit per request
            order='date',  # Sort the search results by date
            pageToken=next_page_token,
            type='video'
        ).execute()

        for search_result in search_response.get('items', []):
            video_id = search_result['id']['videoId']
            video_title = search_result['snippet']['title']
            video_url = f"https://www.youtube.com/watch?v={video_id}"
            channel_title = search_result['snippet']['channelTitle']
            published_at = search_result['snippet']['publishedAt']

            # Get additional details from the videos.list method
            video_response = youtube.videos().list(
                part='statistics',
                id=video_id
            ).execute()

            # Extract the number of comments, if available
            if 'items' in video_response and video_response['items']:
                video_comments = video_response['items'][0]['statistics'].get('commentCount', 'Not available')
            else:
                video_comments = 'Not available'

            all_videos.append({
                'Title': video_title,
                'URL': video_url,
                'Source': channel_title,
                'Published At': published_at,
                'Comments': video_comments
            })

        next_page_token = search_response.get('nextPageToken')
        if not next_page_token or len(all_videos) >= max_results:
            break  # Exit loop if no more pages or reached max_results

    return all_videos[:max_results]

if __name__ == '__main__':
    query = 'Chatgpt'
    max_results_desired = 10000  # Set a reasonable number due to quota limits
    results = youtube_search(query, max_results=max_results_desired)
    df = pd.DataFrame(results)
    filename = f"{query.replace(' ', '_')}_youtube_search_results.csv"
    df.to_csv(filename, index=False, encoding='utf-8')
    print(f"Saved search results to {filename}")



Saved search results to Japan_youtube_search_results.csv
